In [135]:
import tabula
import pdfplumber
import fitz
import camelot
import warnings , math
import time
from collections import Counter

warnings.filterwarnings("ignore", category=UserWarning) 

In [109]:

path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"

samco_path = r"\files\SamcoFactSheet2024.pdf"
tata_path = r"\files\TataFactSheet2024.pdf"

In [ ]:
"""
Basic page cleaning, image removal, page number removal,
repetitive data removal

"""

In [106]:
data = {
    "metadata":{
      
    },
    "content":{
        
    }
}

In [112]:

file_path = path + samco_path
file_path

'C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\mywork-repo\\files\\SamcoFactSheet2024.pdf'

In [ ]:
#Set basic data for json to get
def set_basic_pdf(pdf_path, data):
    with fitz.open(pdf_path) as pdf:
        page_count = pdf.page_count
        file_name = pdf.name
        currtime = time.ctime()
        
        data['metadata'] = {
            "page_count": page_count,
            "file_name": file_name,
            "process_date": currtime
        }
    
    return data

data = set_basic_pdf(file_path, data)

print(data["metadata"])

In [144]:
remove_content =[
    'Mutual fund investments are subject to market risks, read all scheme related documents carefully. ',
    '(An open-ended scheme investing across large cap, midcap and small cap stocks)',
]

In [145]:
document = fitz.open(file_path)
#print(document.page_count)
final_text = []
for pgn in range(document.page_count):
    remove_content.append(str(pgn+1))
    page = document[pgn]
    text = page.get_text('text')
    for txt in text.split("\n"):
        final_text.append(txt)   


counts = Counter(final_text)

In [147]:
counts

Counter({'High': 81,
         'Moderate': 54,
         'Pharmaceuticals & Biotechnology': 43,
         'Capital Markets': 29,
         'Low': 27,
         'Very': 27,
         'Low to': 27,
         'Moderately': 27,
         'RISK-O-METER': 27,
         '': 24,
         'Benchmark': 24,
         'Industrial Products': 24,
         'Consumer Durables': 24,
         '0.56%': 23,
         'Auto Components': 22,
         'Automobiles': 21,
         '0.18%': 21,
         'Returns (%)': 21,
         'Finance': 20,
         '0.17%': 19,
         'Chemicals & Petrochemicals': 17,
         'Electrical Equipment': 17,
         'Realty': 16,
         '0.01%': 16,
         'Since Inception': 16,
         'IT - Software': 15,
         '-0.01%': 15,
         'Last 1 year': 14,
         'Power': 13,
         'Aerospace & Defense': 13,
         'Retailing': 13,
         'Personal Products': 13,
         'Additional': 13,
         'Investors understand that their': 12,
         'principal will be at v

In [146]:
remove_content

['Mutual fund investments are subject to market risks, read all scheme related documents carefully. ',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24']

1